In [86]:
import random

import numpy as np
import pandas as pd
import torchvision.models as models
import torchvision
import torch.nn as nn
import torch

from data_loader import *

In [87]:
device = torch.device('cpu') # training on CPU

In [88]:
# def train_test_split(images, labels, prc = 0.8):
#     all_data = list(range(0, len(images)))
#     train_data = random.sample(all_data, int(len(images) * 0.8))
#     test_data = [el for el in all_data if el not in train_data]

#     return images[train_data], labels[train_data], images[test_data], labels[test_data]

In [89]:
def hybrid_blocks(student, teacher):
    '''
    Function used to get BasicBlocks from ResNet class model
    '''
    student_layers = [student.layer1, student.layer2, student.layer3, student.layer4]
    teacher_layers = [teacher.layer1, teacher.layer2, teacher.layer3, teacher.layer4]

    student_blocks = []
    teacher_blocks = []
    
    for i in range(len(student_layers)):
        teacher_blocks += list(np.array_split(teacher_layers[i], len(student_layers[i]))) # divide teacher blocks into n list, where n is number of student blocks
        student_blocks += [el for el in student_layers[i]]

    return student_blocks, teacher_blocks

In [90]:
def forward(x, student, teacher, a_all):
    '''
    Forward function for hybrid ResNet 
    '''
    def _forward_blocks(x, student_blocks, teacher_blocks, a_all):
        '''
        Forward function containing only hybrid blocks predicitons
        '''
        len_teacher_blocks = len(teacher_blocks)
        len_student_blocks = len(student_blocks)
        assert len_teacher_blocks == len_student_blocks   # check if size of blocks is the same
        tmp_x = x
        for i in range(len_student_blocks): # hybrid block
            if a_all[i] == 1: # student path
                tmp_x = student_blocks[i].forward(tmp_x)

            if a_all[i] == 0: # teacher path
                for j in range(len(teacher_blocks[i])):
                    tmp_x = teacher_blocks[i][j].forward(tmp_x)

        return tmp_x, a_all

    student_blocks, teacher_blocks = hybrid_blocks(student, teacher)
    
    tmp_x = x     # forward pipeline
    tmp_x = student.conv1(tmp_x)
    tmp_x = student.bn1(tmp_x)
    tmp_x = student.relu(tmp_x)
    tmp_x = student.maxpool(tmp_x)
    tmp_x, a_all = _forward_blocks(tmp_x, student_blocks, teacher_blocks, a_all)
    tmp_x = student.avgpool(tmp_x)
    tmp_x = torch.flatten(tmp_x, 1)
    output = student.fc(tmp_x)

    return output

In [103]:
def training(data, student, teacher, p, epochs = 6):
    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.Adam(student.parameters(), lr=0.001)
    train_loss = []
    for e in range(epochs):
        print(f"Epoch no. {e}")
        score = 0
        loss = 0
        student_blocks, teacher_blocks = hybrid_blocks(student, teacher)
        #a_all = [np.random.binomial(1, p) for i in range(len(student_blocks))]   # hybrid block building schema 
        a_all =[1,0,1,1,1,1,1,1]
        for block, a in zip(student_blocks,a_all):
            if a==0:
                for param in block.parameters():
                    param.requires_grad=False
            else:
                for param in block.parameters():
                    param.requires_grad=True      
        for image, label in data:
            student_blocks, teacher_blocks = hybrid_blocks(student, teacher)
            image = image.to(device)
            label = label.to(device)
            optimizer.zero_grad()
            y_pred = forward(image, student, teacher, a_all)
            loss = loss_function(y_pred, label)         
            loss.backward() 
            optimizer.step()
            val, index_ = torch.max(y_pred, axis=1)
            score += torch.sum(index_ == label.data).item()
            loss += loss.item()
            
        epoch_score = score / len(data)
        epoch_loss = loss / len(data)
        train_loss.append(epoch_loss)
        print("Training loss: {}, accuracy: {}".format(epoch_loss, epoch_score))

In [104]:
resnet34 = models.resnet34(pretrained=True)
resnet18 = models.resnet18(pretrained=False)

In [ ]:
training(trainloader, resnet18, resnet34, 0.8)

In [ ]:
#list(resnet18.children())[4]
#print(list(student_blocks[0].parameters())[0])

In [ ]:
# i=0
# for child in student.children():
#     if i==0:
#         i+=1
#         continue
#     if a_all[i-1]:
#                 child.requeirs_grad=False
#             else:
#                 child.requeirs_grad=True
#             i+=1

In [ ]:
# names_34_p=[]
# params_34_p=[]
# print("resnet34")   
# for name, param in resnet34.named_parameters():
#     print(name)
#     names_34_p.append(name)
#     print(param)
#     params_34_p.append(param)
# print(len(names_34_p))
# print(len(params_34_p))

# names_18_p=[]
# params_18_p=[]
# print("\nresnet18")    
# for name, param in resnet18.named_parameters():
#     print(name)
#     names_18_p.append(name)
#     #print(param)
#     params_18_p.append(param)
# print(len(names_18_p))
# print(len(params_18_p))

In [ ]:
#jak sprawdzić czy zadziałało?
# powinno być 
# teacher[0] niezmieniony
#rint(names_34_p[3])
#rint(params_34_p[3])
# student[1] zmieniony 
#rint(names_18_p[12])
#rint(params_18_p[12])
# student_blocks, teacher_blocks = hybrid_blocks(resnet18, resnet34)
# print("\nteacher_blocks[0]")
# print(list(student_blocks[0].parameters())[2])
# print("\n")
# print("\nteacher_blocks[1")
# print(teacher_blocks[1])
# print("\n")
# print("\nteacher_blocks[2")
# print(teacher_blocks[2])
# print("\nteacher_blocks[3)")
# print(teacher_blocks[3])
# print("\nteacher_blocks[4")
# print(teacher_blocks[4])
# print("\nresnet18")
# print("\nstudent_blocks[0]")
# student_blocks[0]
# print("\nstudent_blocks[1")
# print(student_blocks[1])
# print("\nstudent_blocks[2")
# print(student_blocks[2])
# print("\nstudent_blocks[3)")
# print(student_blocks[3])
# print("\nstudent_blocks[4")
# print(student_blocks[4])

In [ ]:
#student_blocks, teacher_blocks = hybrid_blocks(models.resnet18(pretrained=False), models.resnet34(pretrained=False))
#a_all = [np.random.binomial(1, 0.8) for i in range(len(student_blocks))]
#a_all
#student_blocks[1]

In [ ]:
#teacher_blocks[2]
#for p in resnet34.parameters():
#    print(p)

In [105]:
# names_34_p=[]
# params_34_p=[]
# print("resnet34")   
# for name, param in resnet34.named_parameters():
#     names_34_p.append(name)
#     params_34_p.append(param)


# names_18_p=[]
# params_18_p=[]
# print("\nresnet18")    
# for name, param in resnet18.named_parameters():
#     names_18_p.append(name)
#     params_18_p.append(param)
#-------------------------------------------------
print("\nTraining") 
training(trainloader, resnet18, resnet34, 0.8)
#---------------------------------------------------
# names_34_f=[]
# params_34_f=[]
# print("resnet34")   
# for name, param in resnet34.named_parameters():
#     names_34_f.append(name)
#     params_34_f.append(param)


# names_18_f=[]
# params_18_f=[]
# print("\nresnet18")    
# for name, param in resnet18.named_parameters():
#     names_18_f.append(name)
#     params_18_f.append(param)

# print("resnet34")     
# print(names_34_p[3])
# print(params_34_p[3])
# print(names_34_f[3])
# print(params_34_f[3])
# print("resnet34") 
# torch.equal(params_34_p[3],params_34_f[3])
# print("\nresnet18")  
# print(names_18_p[12])
# print(params_18_p[12])
# print(names_18_f[12])
# print(params_18_f[12])
# print("\nresnet18")  
# torch.equal(params_18_p[12],params_18_f[12])


Training
Epoch no. 0
Training loss: 0.005277065094560385, accuracy: 57.25609756097561
Epoch no. 1
Training loss: 0.0017899281810969114, accuracy: 60.329268292682926
Epoch no. 2
Training loss: 0.0010655565420165658, accuracy: 60.86585365853659
Epoch no. 3
Training loss: 0.0016414590645581484, accuracy: 61.25609756097561
Epoch no. 4
Training loss: 0.0033835263457149267, accuracy: 61.353658536585364
Epoch no. 5
Training loss: 0.0009427148615941405, accuracy: 61.46341463414634


In [ ]:
#torch.equal(params_34_p[3],params_34_f[3])
# print("\nresnet34") 
# for i in range(len(params_34_p)):
#     print(torch.equal(params_34_p[i],params_34_f[i]))
# print("\nresnet18") 
# for i in range(len(params_18_p)):
#     print(torch.equal(params_18_p[i],params_18_f[i]))
#torch.equal(params_18_p[12],params_18_f[12])

In [ ]:
# for child in model_ft.children():
# i=0
#         for child in student.children():
#             if i=0:
#                 i+=1
#                 continue
#             if !a_all[i-1]:
#                 child.requeirs_grad=False
    

chciałem sprawdzić czy funkcja trenowania zmienia resnet18 i nie zmienia resnet34 tak jakbyśmy chcieli,
zrobiłem test polegający na:
* zapisaniu parametrów modeli, 
* wywołaniu funkcji trenowania,
* zapisaniu parametrów (wytrenowanych?) modeli,
* porównanie parametrów.\
Okazało się że parametry są takie same, więc albo funkcja trenująca nie trenuje modeli albo ja zrobiłem błąd gdzieś.\
Nie wiedząc co z tym zrobić postanowiłem zająć się czymś innym czyli 